In [1]:
!pip install sentencepiece
import sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from transformers import MarianMTModel, MarianTokenizer
import librosa

In [3]:
# Function to create a speech recognition model
def create_speech_recognition_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Function to create a machine translation model
def create_translation_model(model_name, max_length):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer

# Function to recognize speech and translate it
def speech_to_text_translation(audio_data, speech_model, translation_model, translation_tokenizer, lang1, lang2):

    predicted_text = speech_model.predict(np.expand_dims(audio_data, axis=0))


    source_text = str(predicted_text[0])  # Converting to string
    translated_text = translate_text(source_text, translation_model, translation_tokenizer, lang2)


    return translated_text




In [4]:
# Function to translate text using a pre-trained machine translation model
def translate_text(text, translation_model, tokenizer, lang2):
    # Prepending the target language token to the input text
    text_with_lang = f">{lang2} " + text
    inputs = tokenizer(text_with_lang, return_tensors="pt", padding=True, truncation=True, max_length=512)
    translated_ids = translation_model.generate(**inputs).squeeze()

    # Decoding the token IDs to text
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text




In [ ]:
if __name__ == "__main__":
    # Define parameters
    input_shape = (None, 1)  # Adjusting input shape based on our audio features
    num_classes = 4  # Number of language classes
    translation_model_name = 'Helsinki-NLP/opus-mt-en-fr'  # Adjust the target language

    # Creating and compiling the speech recognition model
    speech_model = create_speech_recognition_model(input_shape, num_classes)
    speech_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Creating and loading the machine translation model
    translation_model, translation_tokenizer = create_translation_model(translation_model_name, max_length=512)

    # Stateless model: getting audio input directly from the user
    print("Please provide the path to the audio file:")
    audio_path = input().strip()
    audio_data, _ = librosa.load(audio_path, sr=16000)  # Adjust sample rate as needed




tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Please provide the path to the audio file:


In [ ]:
 # Performing speech-to-text translation and print the result
    lang1 = 'en'  # we can replace it with the source language selected by the user
    lang2 = 'fr'  # we can replace it with the target language selected by the user
    translated_text = speech_to_text_translation(audio_data, speech_model, translation_model, translation_tokenizer, lang1, lang2)
    print("Translated Text:", translated_text)